In [19]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key

from api_keys import g_key

In [20]:
 # Store csv data into a Pandas df
    
city_info_df = pd.read_csv("output_data/cities.csv")
city_info_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [21]:
# Configure gmaps API key

gmaps.configure(api_key=g_key)

In [22]:
# Humidity heatmap

humidity = city_info_df["Humidity"]
locations = city_info_df[["Lat", "Lng"]]
fig_1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig_1.add_layer(heat_layer)

fig_1

Figure(layout=FigureLayout(height='420px'))

In [23]:
 # Narrow down cities based on criteria and drop any null values
    
city_info_filtered_df = city_info_df.loc[(city_info_df["Max Temp"] < 90) & (city_info_df["Max Temp"] > 75) \
                                    & (city_info_df["Wind Speed"] < 10) \
                                    & (city_info_df["Cloudiness"] == 0)].dropna()
city_info_filtered_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72


In [25]:
# Create DataFrame (hotels_df) to create empty column to store hotel names along with their location information

hotels_df = city_info_filtered_df[["City", "Country", "Lat", "Lng"]].copy()
hotels_df["Hotel Name"] = ""
hotels_df

,City,Country,Lat,Lng,Hotel Name
88,cidreira,BR,-30.17,-50.22,
125,arraial do cabo,BR,-22.97,-42.02,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
142,sarai mir,IN,26.02,82.92,
176,nador,MA,35.17,-2.93,
244,pasni,PK,25.26,63.48,
316,gunjur,GM,13.18,-16.76,
416,kirkuk,IQ,35.47,44.40,
518,bonito,BR,-21.13,-56.49,


In [27]:
# search for a hotel based on specific radius

parameters = {
    "radius": 3000,
    "types": "lodging",
    "key": g_key
}

# Run through df

for index, row in hotels_df.iterrows():
    
    # Pull the latitude and longitutde from the df
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # F string including the latitude and the longitude
    
    parameters["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    
    url_address = requests.get(base_url, params=parameters)
    
    # convert to json
    url_address = url_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotels_df.loc[index, "Hotel Name"] = url_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotels_df

,City,Country,Lat,Lng,Hotel Name
316,gunjur,GM,13.18,-16.76,Kuba Lodge
416,kirkuk,IQ,35.47,44.40,Bayaz Han (Beyaz Han)
518,bonito,BR,-21.13,-56.49,Marruá Hotel
527,tomatlan,MX,19.93,-105.25,El Salvial
543,yabrud,SY,33.97,36.66,المنزل


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotels_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotels_info)
fig_1.add_layer(marker_layer)

# Display figure

fig_1

Figure(layout=FigureLayout(height='420px'))